In [7]:
from melon_clustering import PatternExtractorWithGNN, SENTENCES_DIR, Loader
import pandas as pd

word = 'krass'
sentences = Loader.load_sentences_from_word(word, 'de', use_cache=True)
sentences_stanza = Loader.load_sentences_from_word(word, 'de', preprocessor='stanza', use_cache=True)

for i, y in sentences.items():
    print(i, len(y))

for i, y in sentences_stanza.items():
    print(i, len(y))

krass 4658
krasse 934
krassem 16
krassen 608
krasser 620
krasses 216
krass 4658
krasse 934
krassem 16
krassen 608
krasser 620
krasses 216


In [10]:
import re
# Function to extract pos, xpos, and posxpos strings
def extract_tags(sentence):
    # Regex to match <pos/xpos>(word)
    pattern = r'<([^/]+)/([^>]+)>\(([^)]+)\)'

    pos_list = []
    xpos_list = []
    posxpos_list = []
    word_list = [] # TODO

    for match in re.finditer(pattern, sentence):
        pos, xpos, _ = match.groups()
        pos_list.append(f"<{pos}>")
        if 'ROOT' in pos:
            xpos_list.append(f"<{pos}>")
        else:
            xpos_list.append(f"<{xpos}>")
        posxpos_list.append(f"<{pos}/{xpos}>")

    # Join the lists back into strings
    pos_string = " ".join(pos_list)
    xpos_string = " ".join(xpos_list)
    posxpos_string = " ".join(posxpos_list)

    return pos_string, xpos_string, posxpos_string

# Lists to hold the extracted strings
sentences_stanza_pos = []
sentences_stanza_xpos = []
sentences_stanza_posxpos = []

# Extract from the dictionary and populate the lists
for sentence_collection in sentences_stanza.values():
    for sentence in sentence_collection:
        pos_str, xpos_str, posxpos_str = extract_tags(sentence)
        sentences_stanza_pos.append(pos_str)
        sentences_stanza_xpos.append(xpos_str)
        sentences_stanza_posxpos.append(posxpos_str)

# Print the results
print("sentences_stanza_pos:", sentences_stanza_pos)
print("sentences_stanza_xpos:", sentences_stanza_xpos)
print("sentences_stanza_posxpos:", sentences_stanza_posxpos)


sentences_stanza_pos: ['<PRON> <VERB> <PRON> <ADV> <ROOT> <ADP>', '<PRON> <AUX> <ADJ>', '<PRON> <AUX> <ADV> <ROOT> <SCONJ> <PRON> <ADV> <ADV> <ADV> <VERB> <ADV> <ADV> <PRON> <VERB> <CCONJ>', '<X> <ROOT> <ADV> <PRON> <AUX> <PRON> <ADP> <NOUN>', '<VERB> <ROOT> <CCONJ> <ADP> <PRON> <AUX> <PRON> <ADV> <DET> <NOUN> <ADV> <AUX> <DET> <NOUN>', '<ADP> <PRON> <VERB> <CCONJ> <PRON> <VERB> <PRON> <ADV> <ROOT> <VERB> <ADP> <PRON> <ADP> <DET> <ADJ>', '<PRON> <AUX> <ADV> <ADV> <ADJ> <ROOT> <ADV> <ADJ>', '<PROPN>', '<ADV> <ROOT> <PRON> <AUX> <ADV> <VERB> <PRON> <VERB> <NOUN> <ADV> <PRON> <VERB> <DET> <NOUN> <PRON> <AUX> <PRON> <ADP> <DET> <VERB> <CCONJ> <PRON> <AUX> <ADP>', '<PRON> <AUX> <ADV>', '<PRON> <VERB> <DET> <NOUN> <ADV>', '<ROOT> <PRON> <ADV> <ADV> <PART> <VERB>', '<PRON> <AUX> <ADJ> <NOUN> <CCONJ> <DET> <NOUN> <ADP> <ROOT> <ADP> <PRON>', '<ADV> <AUX> <PRON> <ADV>', '<ADV> <AUX> <ADV> <ADV> <ADJ> <ROOT> <VERB> <CCONJ> <PRON> <VERB>', '<PRON> <PROPN> <VERB> <AUX> <AUX> <ADV>', '<PRON> <VERB> 

In [12]:
from collections import defaultdict

# Function to check if one pattern is a subset of another
def is_subset(pattern1, pattern2):
    return pattern1 in pattern2

# Correcting the approach to capture both broader and more specific patterns
def collect_patterns_correct(strings, min_frequency):
    # Dictionary to store pattern counts and indices
    pattern_counts = defaultdict(lambda: {"count": 0, "indices": []})

    for index, string in enumerate(strings):
        # Split the string into words
        words = string.split()

        # Find all patterns containing <ROOT>
        if "<ROOT>" in words:
            root_index = words.index("<ROOT>")
            # Capture patterns that include at least one word before and up to all the words after <ROOT>
            for i in range(1, root_index + 2):  # For words before <ROOT>
                for j in range(1, len(words) - root_index + 1):  # For words after <ROOT>
                    pattern = " ".join(words[root_index - i + 1 : root_index + j])
                    pattern_counts[pattern]["count"] += 1
                    pattern_counts[pattern]["indices"].append(index)

    # Filter patterns based on frequency
    final_pattern_dict = {pattern: data["indices"] for pattern, data in pattern_counts.items() if data["count"] >= min_frequency}

    # Remove smaller subsets
    patterns_to_remove = set()
    patterns_list = list(final_pattern_dict.keys())

    for i, pattern1 in enumerate(patterns_list):
        for j, pattern2 in enumerate(patterns_list):
            if i != j and final_pattern_dict[pattern1] == final_pattern_dict[pattern2]:
                if is_subset(pattern1, pattern2) and len(pattern1) < len(pattern2):
                    patterns_to_remove.add(pattern1)

    # Remove the smaller patterns from the final dictionary
    for pattern in patterns_to_remove:
        del final_pattern_dict[pattern]

    return final_pattern_dict

sentencesa = [
"Ich hab nur durch Fitness einen <ROOT> Rücken",
"Diese Flüssigkeit hat einen <ROOT> Rücken",
"Diese Flüssigkeit hat einen nicht so <ROOT> Rücken",
"doch einen <ROOT> Verhaltenskodex für Zulieferer"
]
collect_patterns_correct(sentencesa, 2)
# hierarchical_pattern_correct_dict = collect_patterns_correct([i for y in sentences.values() for i in y], 2)
# print(len(hierarchical_pattern_correct_dict))
# hierarchical_pattern_correct_dict = collect_patterns_correct(sentences_stanza_pos, 2)
# print(len(hierarchical_pattern_correct_dict))
# hierarchical_pattern_correct_dict = collect_patterns_correct(sentences_stanza_xpos, 2)
# print(len(hierarchical_pattern_correct_dict))
# hierarchical_pattern_correct_dict = collect_patterns_correct(sentences_stanza_posxpos, 2)
# print(len(hierarchical_pattern_correct_dict))


{'<ROOT>': [0, 1, 2, 3],
 '<ROOT> Rücken': [0, 1, 2],
 'einen <ROOT>': [0, 1, 3],
 'einen <ROOT> Rücken': [0, 1]}

In [ ]:
sentences_stanza_pos

In [ ]:
import json
import pandas as pd
from sklearn.metrics import adjusted_rand_score

def save_clusters_to_file(clusters_dict, filename):
    # Save the clusters to a JSON file
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(clusters_dict, f, ensure_ascii=False, indent=4)

def compute_diff_score(reference_clusters, current_clusters):
    # Flatten both reference and current clusters
    reference_labels = [label for i, cluster in enumerate(reference_clusters) for label in [i] * len(cluster)]
    current_labels = [label for i, cluster in enumerate(current_clusters) for label in [i] * len(cluster)]
    return adjusted_rand_score(reference_labels, current_labels)


results = []

for overlap_threshold in [0.2, 0.4, 0.6, 0.8, 1]:
    extractor = PatternExtractorWithGNN()
    extractor.initialize(sentences_dict, overlap_threshold=overlap_threshold)
    extractor.set_up_digraph()
    edge_index = extractor.build_graph()
    node_features = extractor.initialize_node_features(feature_dim=100)

    gnn_node_embeddings = extractor.train_gnn(edge_index, node_features)

    for sigmoid_steepness in [0.2, 0.5, 1]:
        sentence_embeddings = []
        sentence_list = []

        for morphology, sentences in sentences_dict.items():
            for sentence in extractor.add_start_end_flags_lower(sentences):
                embedding = extractor.generate_sentence_embeddings(gnn_node_embeddings, sentence, morphology, steepness=sigmoid_steepness)
                sentence_embeddings.append(embedding)
                sentence_list.append(sentence)

        sentence_embeddings = np.array(sentence_embeddings)
        clusters = extractor.cluster_embeddings(sentence_embeddings, n_clusters=3)

        # Create a dictionary to store the current clusters
        current_clusters_dict = {}
        for cluster_label in set(clusters):
            current_clusters_dict[int(cluster_label)] = [sentence_list[i] for i in range(len(clusters)) if clusters[i] == cluster_label]

        # Save the current clusters to a file
        clusters_filename = f'clusters_sigmoid_{sigmoid_steepness}_overlap_{overlap_threshold}.json'
        save_clusters_to_file(current_clusters_dict, PLOT_DIR / clusters_filename)

        # Compute the diff score compared to the reference clustering
        current_clusters = [current_clusters_dict[label] for label in sorted(current_clusters_dict.keys())]
        diff_score = compute_diff_score(reference_clusters, current_clusters)

        # Append results to the list for the DataFrame
        results.append({
            "Sigmoid Steepness": sigmoid_steepness,
            "Overlap Threshold": overlap_threshold,
            "Diff Score": diff_score
        })

        # Visualize clusters and save the plot
        reduced_embeddings = extractor.reduce_dimensionality(sentence_embeddings, method='pca')
        extractor.visualize_clusters(
            reduced_embeddings,
            clusters,
            sentence_list,
            plot=False,
            method='pca',
            appendix=f'sigmoid_steepness={sigmoid_steepness}, invgraph_enmeshment={overlap_threshold}'
        )

# Create a DataFrame from the results list
df_results = pd.DataFrame(results)
heatmap_data = df_results.pivot("Sigmoid Steepness", "Overlap Threshold", "Diff Score")

# Plot the heatmap
import matplotlib as plt
import seaborn as sns
plt.figure(figsize=(8, 6))
sns.heatmap(heatmap_data, annot=True, cmap="RdYlGn", cbar=True, center=0.85)
plt.title("Diff Score Heatmap")
plt.show()